<a href="https://colab.research.google.com/github/Thushan97/CURE/blob/master/gpt_conut_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!git clone https://github.com/Thushan97/CURE.git

fatal: destination path 'CURE' already exists and is not an empty directory.


In [21]:
!mv /content/CURE/* /content/

mv: cannot stat '/content/CURE/*': No such file or directory


In [22]:
# pretrain model download
!wget "https://zenodo.org/record/7030145/files/models.tar.xz?download=1" -c -O 'models.tar.xz'
!mkdir /content/data/models
!tar -xf models.tar.xz
!mv /content/models/* /content/data/models/


--2022-09-12 07:58:52--  https://zenodo.org/record/7030145/files/models.tar.xz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

mkdir: cannot create directory ‘/content/data/models’: File exists
^C


In [23]:
!pip install transformers==2.10.0 subword-nmt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


---

In [24]:
import os
import sys
import json
import time
import codecs
import random
import numpy as np
import torch
import torch.nn as nn
from transformers import OpenAIGPTLMHeadModel

GPT_CONUT_TRAINER_DIR = os.path.abspath('/content')#os.path.abspath(__file__)[: os.path.abspath(__file__).rindex('/') + 1]

In [25]:
from src.models.gpt_conut import GPTCoNuTModel
from src.dataloader.dictionary import Dictionary
from src.dataloader.gpt_conut_data_loader import GPTCoNuTDataLoader

In [40]:
# print(f'CUDA GPU availible : {torch.cuda.is_available()}')
# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [42]:
class GPTCoNuTTrainer():
    def __init__(self, train_loader, valid_loader, dictionary, gpt_file):
        gpt_loaded = torch.load(gpt_file)
        config = gpt_loaded['config']
        gpt_model = OpenAIGPTLMHeadModel(config).cuda()
        gpt_model.load_state_dict(gpt_loaded['model'])

        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.dictionary = dictionary

        self.batch_size = 12
        self.load_size = 1200   # load 1200 samples from training data every time

        self.gpt_model = gpt_model
        self.model = None
        self.hyper_parameter = {}
        self.optimizer = None
        self.current_train_step = 0
        self.val_loss = {}

    def shuffle_dataset(self):
        indices = [i for i in range(len(self.train_loader.dataset))]
        random.shuffle(indices)
        return indices

    def train_step(self, samples):
        self.model.train()
        self.current_train_step += 1
        self.optimizer.zero_grad()

        batch = self.train_loader.dataset.collater(samples)
        if torch.cuda.is_available():
            outputs = self.model(
                batch['net_input']['src_tokens'].cuda(),
                batch['net_input']['src_with_prev_context'].cuda(),
                batch['net_input']['ctx_tokens'].cuda(),
                prev_tokens_index=batch['target_index'].cuda(),
                prev_tokens_with_context=batch['target_with_prev_context'].cuda(),
                labels=batch['target'].cuda(),
            )
        else:
            outputs = self.model(
                batch['net_input']['src_tokens'],
                batch['net_input']['src_with_prev_context'],
                batch['net_input']['ctx_tokens'],
                prev_tokens_index=batch['target_index'],
                prev_tokens_with_context=batch['target_with_prev_context'],
                labels=batch['target'],
            )
        logits, avg_attn_scores, apr_loss, lm_loss = outputs[:4]
        loss = apr_loss + 0.3 * lm_loss
        loss.mean().backward()
        nn.utils.clip_grad_norm_(self.model.parameters(), 0.5, norm_type=2)
        self.optimizer.step()
        return loss.mean().item(), apr_loss.mean().item(), lm_loss.mean().item()

    def valid_step(self, samples):
        self.model.eval()
        batch = self.valid_loader.dataset.collater(samples)
        outputs = self.model(
            batch['net_input']['src_tokens'].cuda(),
            batch['net_input']['src_with_prev_context'].cuda(),
            batch['net_input']['ctx_tokens'].cuda(),
            prev_tokens_index=batch['target_index'].cuda(),
            prev_tokens_with_context=batch['target_with_prev_context'].cuda(),
            labels=batch['target'].cuda(),
        )
        logits, avg_attn_scores, apr_loss, lm_loss = outputs[:4]
        loss = apr_loss + 0.3 * lm_loss
        return loss.mean().item(), apr_loss.mean().item(), lm_loss.mean().item(), logits

    def validate_and_save(self, model_id, save_dir):
        oom = 0
        with torch.no_grad():
            val_loss, val_fconv_loss, val_lm_loss = [], [], []
            for i in range(0, self.valid_loader.total_size, self.batch_size):
                samples = [self.valid_loader.dataset[j]
                           for j in range(i, min(len(self.valid_loader.dataset), i + self.batch_size))]
                try:
                    loss, fconv_loss, lm_loss, logits = self.valid_step(samples)
                    val_loss.append(float(loss))
                    val_fconv_loss.append(float(fconv_loss))
                    val_lm_loss.append(float(lm_loss))
                except Exception as e:
                    oom += 1

            info = 'val loss:{}, val apr_loss:{}, val lm_loss:{}, val ppl:{}, oom:{}'.format(
                round(float(np.mean(val_loss)), 6),
                round(float(np.mean(val_fconv_loss)), 6),
                round(float(np.mean(val_lm_loss)), 6),
                round(float(np.exp(np.mean(val_loss))), 6),
                oom
            )
            print(info)

            val_loss = np.mean(val_fconv_loss)
            checkpoint = {
                'model': self.model.state_dict(),
                'optimizer': self.optimizer.state_dict(),
                'current_step': self.current_train_step,
                # 'config': self.model.module.config(),
                'val_loss': val_loss,
            }
            torch.save(checkpoint, save_dir + 'gpt_conut_' + str(model_id) + '.pt')
            self.val_loss[model_id] = {
                'val_loss': val_loss,
                'hyper-parameter': str(self.hyper_parameter),
            }

        return val_loss

    def train(self, model_id, epochs, hyper_parameter, save_dir):
        self.hyper_parameter = hyper_parameter
        self.model = GPTCoNuTModel(
            self.dictionary, embed_dim=384, max_positions=1024,
            src_encoder_convolutions=self.hyper_parameter['src_encoder_convolutions'],
            ctx_encoder_convolutions=self.hyper_parameter['ctx_encoder_convolutions'],
            decoder_convolutions=self.hyper_parameter['decoder_convolutions'],
            dropout=self.hyper_parameter['dropout'], embed_model=self.gpt_model,
        ).cuda()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=6.25e-5)
        # self.model = nn.DataParallel(self.model, device_ids=device_ids)
        
        self.valid_loader.load_data(0, self.valid_loader.total_size)
        for epoch in range(epochs):
            start_time = time.time()
            for i in range(0, self.train_loader.total_size, self.load_size):
                oom = 0
                self.train_loader.load_data(i, i + self.load_size)
                indices = self.shuffle_dataset()
                train_loss, train_apr_loss, train_lm_loss = [], [], []

                start, end = 0, 0
                samples = []
                max_src, max_ctx, max_tgt = 0, 0, 0
                while end < len(self.train_loader.dataset):
                    sample = self.train_loader.dataset[indices[end]]
                    if max_ctx + len(sample['target']) >= 1023 \
                            or max_tgt + len(sample['prev_context']) >= 1023 \
                            or max_ctx + len(sample['source']) >= 1023 \
                            or max_src + len(sample['prev_context']) >= 1023 \
                            or end - start == self.batch_size:
                        try:
                            loss, apr_loss, lm_loss = self.train_step(samples)
                            train_loss.append(loss)
                            train_apr_loss.append(apr_loss)
                            train_lm_loss.append(lm_loss)
                        except Exception as e:
                            oom += 1

                        start = end
                        max_src, max_ctx, max_tgt = 0, 0, 0
                        samples = []
                        continue
                    max_src = max(max_src, len(sample['source']))
                    max_ctx = max(max_ctx, len(sample['prev_context']))
                    max_tgt = max(max_tgt, len(sample['target']))
                    end += 1
                    samples.append(sample)
                if len(samples) > 0:
                    try:
                        loss, apr_loss, lm_loss = self.train_step(samples)
                        train_loss.append(loss)
                        train_apr_loss.append(apr_loss)
                        train_lm_loss.append(lm_loss)
                    except Exception as e:
                        oom += 1

                if (i // self.load_size) % 10 == 0:
                    info = 'epoch:{}, load data:{}, lr:{}, loss:{}, apr_loss:{}, lm_loss:{}, time:{}s, oom:{}'.\
                        format(epoch + 1, i + self.load_size,
                               round(self.optimizer.param_groups[0]['lr'], 10),
                               round(float(np.mean(train_loss)), 6),
                               round(float(np.mean(train_apr_loss)), 6),
                               round(float(np.mean(train_lm_loss)), 6),
                               int(time.time() - start_time), oom
                               )
                    start_time = time.time()
                    print(str(model_id) + ' ' + info)

                if (i // self.load_size) % 100 == 0:
                    self.validate_and_save(model_id, save_dir)
        self.validate_and_save(model_id, save_dir)

In [43]:
if __name__ == '__main__':
    device_ids = [0, 1, 2, 3]
    os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
    
    vocab_file = GPT_CONUT_TRAINER_DIR + '/data/vocabulary/vocabulary.txt'
    train_file = GPT_CONUT_TRAINER_DIR + '/data/data/training_bpe.txt'
    valid_file = GPT_CONUT_TRAINER_DIR + '/data/data/validation_bpe.txt'
    gpt_file = GPT_CONUT_TRAINER_DIR + '/data/models/code_gpt.pt'

    dictionary = Dictionary(vocab_file, min_cnt=0)
    print('dictionary initialized, vocab size:{}'.format(len(dictionary)))

    train_loader = GPTCoNuTDataLoader(train_file, dictionary)
    valid_loader = GPTCoNuTDataLoader(valid_file, dictionary)
    print('data loader initialized, train size:{}, validate size:{}'.
          format(train_loader.total_size, valid_loader.total_size))

    trainer = GPTCoNuTTrainer(train_loader, valid_loader, dictionary, gpt_file)

    hyper_parameter = {
        'src_encoder_convolutions': ((192, 5),) * 1,
        'ctx_encoder_convolutions': ((384, 5),) * 1,
        'decoder_convolutions': ((192, 5),) * 1,
        'dropout': 0.1,
    }
    model_id = 1
    epochs = 5
    trainer.train(model_id, epochs, hyper_parameter, save_dir=GPT_CONUT_TRAINER_DIR + '/data/models/')

dictionary initialized, vocab size:50061
data loader initialized, train size:2000, validate size:100


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


1 epoch:1, load data:1200, lr:6.25e-05, loss:9.608289, apr_loss:7.221054, lm_loss:7.957447, time:54s, oom:44
val loss:5.09603, val apr_loss:3.245551, val lm_loss:6.168262, val ppl:163.371957, oom:0
1 epoch:2, load data:1200, lr:6.25e-05, loss:3.710341, apr_loss:3.077762, lm_loss:2.108595, time:53s, oom:44
val loss:2.526983, val apr_loss:2.283755, val lm_loss:0.81076, val ppl:12.515687, oom:0
1 epoch:3, load data:1200, lr:6.25e-05, loss:2.74184, apr_loss:2.456163, lm_loss:0.952255, time:56s, oom:42
val loss:1.95275, val apr_loss:1.736616, val lm_loss:0.720448, val ppl:7.048044, oom:0
1 epoch:4, load data:1200, lr:6.25e-05, loss:2.106179, apr_loss:1.856535, lm_loss:0.832149, time:55s, oom:47
val loss:1.574926, val apr_loss:1.362139, val lm_loss:0.709291, val ppl:4.830386, oom:0
1 epoch:5, load data:1200, lr:6.25e-05, loss:1.882184, apr_loss:1.634849, lm_loss:0.824448, time:56s, oom:38
val loss:1.398118, val apr_loss:1.191759, val lm_loss:0.687861, val ppl:4.047574, oom:0
val loss:1.35428